In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pwd
%cd '/content/drive/MyDrive/langchain'
%pwd

/content/drive/MyDrive/langchain


'/content/drive/MyDrive/langchain'

In [ ]:
!pip install langchainhub
!pip install -q streamlit
!pip install pyngrok
!npm install localtunnel
!pip -q install xformers einops
!pip -q install --upgrade together
!pip -q install -U FlagEmbedding
!pip -q install huggingface_hub tiktoken
!pip -q install chromadb
!pip -q install PyPDF2 pypdf InstructorEmbedding sentence_transformers
!pip -q install langchain openai
!pip install cohere
!pip install -q datasets loralib sentencepiece
!pip install unstructured
!pip install duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/drive/MyDrive/langchain/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/drive/MyDrive/langchain/package.json'
npm WARN langchain No description
npm WARN langchain No repository field.
npm WARN langchain No README data
npm WARN langchain No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 4.661s

3 packages are looking for funding
  run `npm fund` for details

found 5 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details

In [ ]:
!python together_app_api.py

In [ ]:
%%writefile app.py

import streamlit as st
from PIL import Image
import base64
import requests
import logging
from io import StringIO
from io import BytesIO
import ast
from together_app_api import agent_executor,memory,OpenAI,LLMSummarizationCheckerChain,generate_queries,multi_qa,ListHandler
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
import urllib.request
import urllib
import time
import logging
from io import StringIO

# 네이버 파파고 api 예시 그대로 따라 하시고  발급받은 clietn id와 secret 설정 하면 됩니다
client_id = "" # 개발자센터에서 발급받은 Client ID 값
client_secret = "" # 개발자센터에서 발급받은 Client Secret 값
url = ""

flag =True

def papago(content,data):

    korText = urllib.parse.quote(content)
    time.sleep(0.001)

    data = data + korText

    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
    request.add_header("X-NCP-APIGW-API-KEY", client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()

    if (rescode == 200):

        response_body = response.read()
        time.sleep(0.001)

        return ast.literal_eval(response_body.decode('utf-8'))["message"]["result"]["translatedText"]

    else:
        print("Error Code:" + rescode)


api_key = "" # your open ai api key


st.sidebar.title("나만의 다큐(Document)")

global tuple_data
tuple_data=["multi_query_ready"]

user_input = st.text_input("메시지를 입력하세요")

if 'session_state' not in st.session_state:
    st.session_state.session_state = {"selected_text": None, "tuple_data": []}


if st.sidebar.button("전송"):

    try:

      log_list = []
      text_list=[]
      handler = ListHandler(log_list)

      logger = logging.getLogger()
      logger.addHandler(handler)
      logger.setLevel(logging.INFO)

      unique_docs=multi_qa.get_relevant_documents(query=user_input)

      s=log_list[1]

      queries = s[s.find("[")+1:s.find("]")].split(", ")

      # 딕셔너리로 변환
      query_dict = {}
      for query in queries:
          key, value = query.split(". ", 1)
          query_dict[key] = value.strip("'")

      text_list = list(query_dict.values())
      data="source=ko&target=en&text="
      user_input = papago(user_input,data)

      #tuple_data = generate_queries(user_input)

      #tuple_data=list(tuple_data)


      tuple_data=text_list


      for i in range(len(tuple_data)):
          data="source=en&target=ko&text="
          tuple_data[i]=papago(str(tuple_data[i]),data)

      #  tuple_data를 session_state에 저장
      st.session_state.session_state["tuple_data"] = tuple_data

      bot_response = agent_executor.invoke({"input": user_input})["output"]

      data="source=en&target=ko&text="

      bot_response=papago(bot_response,data)

      st.write(bot_response)


    except Exception:

      flag=False
      data="source=ko&target=en&text="
      user_input = papago(user_input,data)
      bot_response = agent_executor.invoke({"input": user_input})["output"]
      bot_response=papago(bot_response,data)
      bot_response+="\n\n duckduckgo search \n\n"
      st.write(bot_response)



if flag:
  selected_text = st.selectbox("연관된 질문을 선택하세요", tuple(st.session_state.session_state["tuple_data"]))


if st.sidebar.button("새 질문 적용 하기"):
    data="source=ko&target=en&text="
    selected_text_=papago(selected_text,data)


    bot_response = agent_executor.invoke({"input": selected_text_})["output"]
    data="source=en&target=ko&text="

    bot_response=papago(bot_response,data)

    data="source=en&target=ko&text="

    bot_response=papago(bot_response,data)
    st.write(bot_response)



uploaded_file = st.file_uploader("이미지 파일을 업로드하세요", type=["png", "jpg"])
user_img_input = st.text_input("이미지에 대한 메시지를 입력하세요")





if st.sidebar.button("사진전송"):
    data="source=ko&target=en&text="
    user_img_input = papago(user_img_input,data)

    if uploaded_file is not None:
        img_path = "./data/img_file/"
        img_path+=str(uploaded_file.name)

        image = Image.open(uploaded_file)
        st.image(image, caption='업로드된 이미지.',use_column_width=True)

        image.save(img_path)

        def encode_image(img_path):

            with open(img_path, "rb") as image_file:
                return base64.b64encode(image_file.read()).decode('utf-8')

        base64_image = encode_image(img_path)


        headers ={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"

        }

        payload = {
            "model": "gpt-4-vision-preview",
            "messages": [

                {
                    "role": "user",
                    "content": [
                        {

                            "type": "text",
                            "text": user_img_input
                        },
                        {
                            "type": "image_url",
                            "image_url":
                            {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 500
        }
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        json_response = response.json()
        content = json_response['choices'][0]['message']['content']

        data="source=en&target=ko&text="
        content=papago(content,data)
        st.write(content)


# 끝내기 버튼
if st.sidebar.button("끝내기"):
    st.sidebar.text("지금까지 내용 요약")
    llm = OpenAI(temperature=0,max_tokens=1000)
    checker_chain = LLMSummarizationCheckerChain.from_llm(llm, verbose=False, max_checks=2)
    text=""""""

    for i in range(len(memory.buffer)):
        text+=memory.buffer[i].content
        text+="\n\n"

    response = checker_chain.run(text)

    data="source=en&target=ko&text="
    response = papago(response,data)


    st.write(response)


Overwriting app.py


In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 35.197.122.12


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.827s
your url is: https://wet-pants-begin.loca.lt
^C
